# Grating Coupler Inverse Design

Gradient-based topology optimization of a silicon photonic grating coupler
using the Hyperwave Community SDK.

**Workflow:**
1. Define SOI layer stack and rectangular design region
2. Build lightweight structure recipe (cloud reconstructs the 3D structure)
3. Create Gaussian beam source on cloud GPU
4. Run forward simulation to verify setup
5. Optimize on cloud GPU with adjoint gradients
6. Visualize optimized fields and coupling efficiency

All heavy computation (FDTD, mode solving, gradient computation) runs on cloud
GPUs. Only lightweight operations (2D array filtering, plotting) run locally,
so this notebook works on any machine -- including free-tier Colab.

> **Note:** This notebook demonstrates the inverse design workflow as a preview.
> The optimized design is not yet fabrication-ready -- binarization penalties and
> fabrication constraints (minimum feature size enforcement, etch rule checks)
> are still in development. GDS export should only be used after these
> post-processing steps are applied.

In [ ]:
%pip install "hyperwave-community[gds] @ git+https://github.com/spinsphotonics/hyperwave-community.git@feb-inverse-design-notebook" -q

In [ ]:
import hyperwave_community as hwc
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time

hwc.configure_api(api_key="your-api-key-here")

## 1. Physical Parameters

Standard 220nm SOI at 1550nm with partial-etch grating coupler.


In [ ]:
import math

# Materials
n_si = 3.48
n_sio2 = 1.44
n_clad = 1.44
n_air = 1.0

# Wavelength
wavelength_um = 1.55

# Layer thicknesses (um)
h_dev = 0.220        # total silicon device layer
etch_depth = 0.110   # partial etch depth
h_box = 2.0          # buried oxide
h_clad = 0.78        # SiO2 cladding
h_sub = 0.8          # silicon substrate
h_air = 1.0          # air above cladding
pad = 3.0            # absorber padding (top and bottom)

# Grid resolution
dx = 0.035           # 35nm structure grid
pixel_size = dx / 2  # 17.5nm theta grid (2x for subpixel averaging in epsilon())
domain = 40.0        # um total domain

# Waveguide
wg_width = 0.5       # um
wg_length = 5.0      # um

# Fiber
beam_waist = 5.2     # um (SMF-28 mode field radius at 1550nm)
fiber_angle = 14.5   # degrees from vertical (standard SMF coupling angle)

# Structure grid dimensions (35nm)
Lx = int(domain / dx)
Ly = Lx

# Theta grid dimensions (17.5nm, 2x structure)
theta_Lx = 2 * Lx
theta_Ly = 2 * Ly

# Layer thicknesses in pixels (FLOAT for subpixel averaging -- critical for
# thin layers like etch where rounding shifts interfaces by up to 0.5 cells)
h_p = pad / dx
h0 = h_air / dx
h1 = h_clad / dx
h2 = etch_depth / dx
h3 = (h_dev - etch_depth) / dx
h4 = h_box / dx
h5 = h_sub / dx
Lz = int(math.ceil(h_p + h0 + h1 + h2 + h3 + h4 + h5 + h_p))

# Key Z positions (rounded to nearest pixel for array indexing)
z_etch = int(round(h_p + h0 + h1))
z_slab = z_etch + int(round(h2))
z_box = z_slab + int(round(h3))

# Frequency
wl_px = wavelength_um / dx
freq = 2 * np.pi / wl_px
freq_band = (freq, freq, 1)

# Permittivity
eps_si = n_si**2
eps_sio2 = n_sio2**2
eps_clad = n_clad**2
eps_air = n_air**2

# Density filtering (matches reference GC design)
DENSITY_RADIUS = 3
DENSITY_ALPHA = 0.8
DESIGN_LAYER = 3  # etch layer index

print(f"Structure grid: {Lx} x {Ly} x {Lz} ({dx * 1000:.0f} nm)")
print(f"Theta grid: {theta_Lx} x {theta_Ly} ({pixel_size * 1000:.1f} nm)")
print(f"Layers (px): pad={h_p:.2f} air={h0:.2f} clad={h1:.2f} "
      f"etch={h2:.2f} slab={h3:.2f} BOX={h4:.2f} sub={h5:.2f} pad={h_p:.2f}")
print(f"Fiber angle: {fiber_angle} deg")

## 2. Design Variables (Theta)

`theta` controls the etch layer (top 110nm of 220nm Si):
`theta=1` = unetched silicon, `theta=0` = etched to cladding.

The bottom 110nm (dev layer) is solid silicon wherever theta > 0, forming a fan
shape under the grating. This matches the reference GC structure.

Theta operates at 2x the structure resolution (17.5nm vs 35nm). Density filtering
with `radius=3, alpha=0.8` enforces minimum feature size.

Fixed waveguide on the left, rectangular design region initialized to 0.5.

In [ ]:
# Waveguide in structure pixels (35nm grid)
wg_len = int(round(wg_length / dx))
wg_hw = int(round(wg_width / 2 / dx))

# Waveguide in theta pixels (17.5nm grid, 2x structure)
wg_len_theta = int(round(wg_length / pixel_size))
wg_hw_theta = int(round(wg_width / 2 / pixel_size))

# Rectangular design region (in theta pixels)
abs_margin = 80  # structure pixels
abs_margin_theta = 2 * abs_margin
design_region = {
    'x_start': wg_len_theta,
    'x_end': theta_Lx - abs_margin_theta,
    'y_start': abs_margin_theta,
    'y_end': theta_Ly - abs_margin_theta,
}

# Build theta at 2x resolution (17.5nm)
theta_init = np.zeros((theta_Lx, theta_Ly), dtype=np.float32)
theta_init[:wg_len_theta, theta_Ly // 2 - wg_hw_theta : theta_Ly // 2 + wg_hw_theta] = 1.0
dr = design_region
theta_init[dr['x_start']:dr['x_end'], dr['y_start']:dr['y_end']] = 0.5

# Plot
fig, ax = plt.subplots(figsize=(8, 8))
extent = [0, theta_Lx * pixel_size, 0, theta_Ly * pixel_size]
ax.imshow(theta_init.T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
rect = Rectangle(
    (dr['x_start'] * pixel_size, dr['y_start'] * pixel_size),
    (dr['x_end'] - dr['x_start']) * pixel_size,
    (dr['y_end'] - dr['y_start']) * pixel_size,
    linewidth=2, edgecolor='lime', facecolor='none', linestyle='--',
)
ax.add_patch(rect)
ax.set_xlabel('x (um)')
ax.set_ylabel('y (um)')
ax.set_title('Initial Theta (design region outlined)')
plt.colorbar(ax.images[0], ax=ax, label='theta')
plt.tight_layout()
plt.show()

print(f"Theta shape: {theta_init.shape} ({pixel_size * 1000:.1f} nm)")
print(f"Design region: {(dr['x_end'] - dr['x_start']) * pixel_size:.1f} x "
      f"{(dr['y_end'] - dr['y_start']) * pixel_size:.1f} um")

## 3. Layer Stack and Structure Recipe

8-layer SOI stack matching the reference GC design. Two design layers:
- **etch** (top 110nm): grating pattern from theta
- **dev** (bottom 110nm): solid silicon fan wherever theta > 0

`recipe_from_params()` creates a lightweight recipe dict -- no 3D array is built
locally. The cloud GPU reconstructs the full permittivity on each simulation call.

Layers (top to bottom): pad (air, 3um) | air (1um) | cladding (SiO2, 0.78um) |
**etch (SiO2/Si, 0.11um)** | **dev (SiO2/Si, 0.11um)** | BOX (SiO2, 2um) |
substrate (Si, 0.8um) | pad (Si, 3um)

In [ ]:
# Density filtering (matches reference: radius=3, alpha=0.8, pad_width=0)
density_top = hwc.density(theta=jnp.array(theta_init), pad_width=0, alpha=DENSITY_ALPHA, radius=DENSITY_RADIUS)

# Bottom layer: solid silicon wherever theta > 0 (fan shape under grating)
theta_bottom = np.where(theta_init > 0, 1.0, 0.0).astype(np.float32)
density_bottom = hwc.density(theta=jnp.array(theta_bottom), pad_width=0, alpha=DENSITY_ALPHA, radius=DENSITY_RADIUS)

# Uniform slab pattern for cladding/air/substrate layers
slab_density = hwc.density(jnp.zeros((theta_Lx, theta_Ly)), pad_width=0)

# Build structure recipe (lightweight dict -- cloud reconstructs 3D permittivity)
recipe = hwc.recipe_from_params(
    grid_shape=density_top.shape,
    layers=[
        {'density': np.array(slab_density),   'permittivity': eps_air,              'thickness': h_p},
        {'density': np.array(slab_density),   'permittivity': eps_air,              'thickness': h0},
        {'density': np.array(slab_density),   'permittivity': eps_clad,             'thickness': h1},
        {'density': np.array(density_top),    'permittivity': (eps_clad, eps_si),   'thickness': h2},
        {'density': np.array(density_bottom), 'permittivity': (eps_clad, eps_si),   'thickness': h3},
        {'density': np.array(slab_density),   'permittivity': eps_sio2,             'thickness': h4},
        {'density': np.array(slab_density),   'permittivity': eps_si,               'thickness': h5},
        {'density': np.array(slab_density),   'permittivity': eps_si,               'thickness': h_p},
    ],
    vertical_radius=0,
)

Lz_recipe = recipe['metadata']['final_shape'][3]
z_dev = z_etch + int(h2 // 2)

# Layer stack visualization
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(14, 5))

extent = [0, theta_Lx * pixel_size, 0, theta_Ly * pixel_size]
ax0.imshow(np.array(density_top).T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
rect = Rectangle(
    (dr['x_start'] * pixel_size, dr['y_start'] * pixel_size),
    (dr['x_end'] - dr['x_start']) * pixel_size,
    (dr['y_end'] - dr['y_start']) * pixel_size,
    linewidth=2, edgecolor='lime', facecolor='none', linestyle='--',
)
ax0.add_patch(rect)
ax0.set_xlabel('x (um)')
ax0.set_ylabel('y (um)')
ax0.set_title('Filtered density (etch layer)')

ax1.imshow(np.array(density_bottom).T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
ax1.set_xlabel('x (um)')
ax1.set_ylabel('y (um)')
ax1.set_title('Bottom layer (solid fan)')

plt.tight_layout()
plt.show()

print(f"Structure grid: {Lx} x {Ly} x {Lz_recipe} ({dx * 1000:.0f} nm)")
print(f"Theta grid: {theta_Lx} x {theta_Ly} ({pixel_size * 1000:.1f} nm)")
print(f"Etch z={z_etch} ({z_etch * dx:.3f} um), slab z={z_slab} ({z_slab * dx:.3f} um)")

## 4. Source

Unidirectional Gaussian beam via the wave equation error method. The FDTD
simulation runs in free space on a cloud GPU, and the wave equation error is
computed locally to produce a clean downward-propagating beam at 14.5 degrees
from vertical, simulating fiber illumination. Negative `theta` tilts the beam
toward the waveguide (in the -x direction).

In [ ]:
# Source position: in the air gap, 50nm above cladding surface (matches reference)
source_above_surface_um = 0.05
source_z = int(round((pad + h_air - source_above_surface_um) / dx))

# Grating center in structure pixels (convert from theta pixel design region)
grating_x = int(round((dr['x_start'] + dr['x_end']) / 2 * pixel_size / dx))
grating_y = Ly // 2
waist_px = beam_waist / dx

# Generate Gaussian source on cloud GPU (runs FDTD in free space remotely)
source_field, input_power = hwc.generate_gaussian_source(
    sim_shape=(Lx, Ly, Lz),
    frequencies=np.array([freq]),
    source_pos=(grating_x, grating_y, source_z),
    waist_radius=waist_px,
    theta=-fiber_angle,  # negative tilts beam toward waveguide (-x direction)
    phi=0.0,             # tilt in XZ plane
    polarization='y',
    max_steps=5000,
    gpu_type="B200",
)

source_offset = (grating_x - Lx // 2, grating_y - Ly // 2, source_z)
input_power = float(np.mean(input_power))

# Plot source |Ey| and |Hx|
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, idx, name in [(axes[0], 1, '|Ey|'), (axes[1], 3, '|Hx|')]:
    im = ax.imshow(np.abs(np.array(source_field[0, idx, :, :, 0])).T,
                   origin='lower', cmap='hot', extent=[0, Lx * dx, 0, Ly * dx])
    ax.set_xlabel('x (um)')
    ax.set_ylabel('y (um)')
    ax.set_title(f'Source {name}')
    plt.colorbar(im, ax=ax)
plt.suptitle(f'Gaussian source (waist = {beam_waist} um, tilt = {fiber_angle} deg)', fontsize=13)
plt.tight_layout()
plt.show()

print(f"Source shape: {source_field.shape}")
print(f"Source offset: {source_offset}")
print(f"Input power: {input_power:.6f}")

## 5. Forward Simulation (Setup Verification)

Run a forward simulation with the initial design to verify:
- Source illuminates the design region correctly
- Light propagates toward the waveguide
- Layer stack and monitors are correctly placed

This costs 1 GPU simulation but catches setup errors before optimization.


In [ ]:
# Absorber (auto-scaled for resolution)
abs_params = hwc.get_optimized_absorber_params(
    resolution_nm=dx * 1000,
    wavelength_um=wavelength_um,
    structure_dimensions=(Lx, Ly, Lz),
)
abs_widths = abs_params['absorption_widths']
abs_coeff = abs_params['absorber_coeff']

# Set up monitors for field extraction
monitors = hwc.MonitorSet()
output_x = abs_widths[0] + 10

# XY slice at device layer
monitors.add(hwc.Monitor(shape=(Lx, Ly, 1), offset=(0, 0, z_dev)), name='xy_device')
# XZ slice at y=center
monitors.add(hwc.Monitor(shape=(Lx, 1, Lz), offset=(0, Ly // 2, 0)), name='xz_center')
# YZ slice at x=center
monitors.add(hwc.Monitor(shape=(1, Ly, Lz), offset=(Lx // 2, 0, 0)), name='yz_center')
# Waveguide output
monitors.add(hwc.Monitor(shape=(1, Ly, Lz), offset=(output_x, 0, 0)), name='wg_output')

# Run forward simulation using the lightweight recipe
fwd_results = hwc.simulate(
    structure_recipe=recipe,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    monitors_recipe=monitors.recipe,
    absorption_widths=abs_widths,
    absorption_coeff=abs_coeff,
    gpu_type="B200",
)

print(f"Forward sim complete: {fwd_results['sim_time']:.1f}s GPU time")

In [ ]:
# Extract fields and plot |E|^2 cross-sections
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, name, title, ext, xlabel, ylabel in [
    (axes[0], 'xy_device', f'|E|^2 XY (z = {z_dev * dx:.2f} um)',
     [0, Lx * dx, 0, Ly * dx], 'x (um)', 'y (um)'),
    (axes[1], 'xz_center', '|E|^2 XZ (y = center)',
     [0, Lx * dx, 0, Lz * dx], 'x (um)', 'z (um)'),
    (axes[2], 'yz_center', '|E|^2 YZ (x = center)',
     [0, Ly * dx, 0, Lz * dx], 'y (um)', 'z (um)'),
]:
    field = np.array(fwd_results['monitor_data'][name])
    E2 = np.sum(np.abs(field[0, 0:3])**2, axis=0).squeeze()
    vmax = np.percentile(E2, 95) * 4
    im = ax.imshow(E2.T, origin='lower', cmap='hot', extent=ext,
                   aspect='auto', vmax=vmax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.colorbar(im, ax=ax, extend='max')

plt.suptitle('Forward simulation (initial design)', fontsize=14)
plt.tight_layout()
plt.show()

# Check power at waveguide output
wg_field = np.array(fwd_results['monitor_data']['wg_output'])
S = hwc.S_from_slice(jnp.mean(jnp.array(wg_field), axis=2))
power = float(jnp.abs(jnp.sum(S[:, 0, :, :], axis=(1, 2))))
print(f"Waveguide output power: {power:.6f}")
print(f"Coupling (approx): {power / input_power * 100:.1f}%")


## 6. Optimization Setup

### How adjoint-method inverse design works

Each optimization step runs **two FDTD simulations** on the cloud GPU:

1. **Forward solve:** Inject the Gaussian source, propagate through the current
   structure, extract the field at the **loss monitor** (waveguide output).
2. **Adjoint solve:** Inject the gradient of the loss (w.r.t. the loss monitor
   field) back into the simulation *at the loss monitor position*, propagate
   again, and extract the adjoint field at the **design monitor**.

The permittivity gradient is then:

$$\frac{\partial L}{\partial \varepsilon} = \omega \cdot \text{Re}(E_\text{fwd} \cdot E_\text{adj})$$

computed **only inside the design monitor volume** -- not the entire 3D domain.
This is memory-efficient: instead of differentiating through the full
`(Lx, Ly, Lz)` grid, we only need fields in the thin design region (the etch
layer, ~3 pixels in z). The gradient is zero outside this volume since only the
etch layer permittivity depends on theta.

### Structure spec

The `structure_spec` provides the layer stack template (materials, thicknesses,
filter parameters) so the GPU can reconstruct permittivity from any theta.
On each optimization step, the cloud runs the full differentiable chain:
`theta -> density_filter -> permittivity -> FDTD -> loss`.

### Power loss

We use the built-in Poynting vector power loss (`power_axis=0`), which maximizes
the total optical power flowing in the +x direction through the loss monitor
(waveguide output cross-section). This is simpler than mode overlap -- no
pre-computed waveguide mode is needed -- and gives good results for single-mode
waveguides where nearly all coupled power is in the fundamental mode.

In [ ]:
# Structure spec for optimization: tells the GPU how to rebuild permittivity.
# Only the etch layer (index 3) is optimized; all other layers are fixed.
# The dev layer (index 4) uses a static density_bottom pattern.
structure_spec = {
    'layers_info': [
        {'permittivity_values': float(eps_air),              'layer_thickness': float(h_p), 'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': float(eps_air),              'layer_thickness': float(h0),  'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': float(eps_clad),             'layer_thickness': float(h1),  'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': [float(eps_clad), float(eps_si)], 'layer_thickness': float(h2),  'density_radius': DENSITY_RADIUS, 'density_alpha': DENSITY_ALPHA},
        {'permittivity_values': [float(eps_clad), float(eps_si)], 'layer_thickness': float(h3),  'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': float(eps_sio2),             'layer_thickness': float(h4),  'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': float(eps_si),               'layer_thickness': float(h5),  'density_radius': 0, 'density_alpha': 0},
        {'permittivity_values': float(eps_si),               'layer_thickness': float(h_p), 'density_radius': 0, 'density_alpha': 0},
    ],
    'construction_params': {'vertical_radius': 0},
}

# Loss monitor: waveguide output cross-section
loss_monitor_shape = (1, Ly, Lz)
loss_monitor_offset = (output_x, 0, 0)

# Design monitor: etch layer volume (where gradients are computed)
design_monitor_shape = (Lx, Ly, int(round(h2)))
design_monitor_offset = (0, 0, z_etch)

# Waveguide mask at theta resolution (forces theta=1 in waveguide region)
waveguide_mask = np.zeros((theta_Lx, theta_Ly), dtype=np.float32)
waveguide_mask[:wg_len_theta, theta_Ly // 2 - wg_hw_theta : theta_Ly // 2 + wg_hw_theta] = 1.0

# Optimizer settings
NUM_STEPS = 5          # increase to 50-100 for production runs
LR = 0.01
GRAD_CLIP = 0.5

print(f"Loss monitor at x={loss_monitor_offset[0]} ({loss_monitor_offset[0] * dx:.1f} um)")
print(f"Design monitor: {design_monitor_shape} ({design_monitor_shape[2] * dx * 1000:.0f} nm etch layer)")
print(f"Loss type: Poynting vector power (Sx) -- maximize power into waveguide")
print(f"Optimizer: Adam, LR={LR}, {NUM_STEPS} steps")

## 7. Optimization Loop

`run_optimization()` runs the full loop on a cloud GPU and streams results
after each step. Interrupting the kernel cancels the GPU task
immediately -- you are only charged for completed steps.

In [ ]:
# Run optimization on cloud GPU
# Each step streams back as it completes. Interrupt kernel to stop early.
results = []

for step_result in hwc.run_optimization(
    theta=theta_init,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    structure_spec=structure_spec,
    loss_monitor_shape=loss_monitor_shape,
    loss_monitor_offset=loss_monitor_offset,
    design_monitor_shape=design_monitor_shape,
    design_monitor_offset=design_monitor_offset,
    power_axis=0,              # maximize Poynting vector Sx (power into WG)
    power_maximize=True,
    waveguide_mask=waveguide_mask,
    num_steps=NUM_STEPS,
    learning_rate=LR,
    grad_clip_norm=GRAD_CLIP,
    absorption_widths=abs_widths,
    absorption_coeff=abs_coeff,
    gpu_type="B200",
):
    results.append(step_result)
    loss = step_result['loss']
    print(f"Step {step_result['step']:3d}/{NUM_STEPS}:  power = {abs(loss):.6f}  "
          f"|grad|_max = {step_result['grad_max']:.3e}  ({step_result['step_time']:.1f}s)")

# Summary
powers = [abs(r['loss']) for r in results]
best_idx = int(np.argmax(powers))
best_power = powers[best_idx]
best_eff = best_power / input_power * 100
print(f"\nBest: {best_eff:.2f}% ({-10 * np.log10(max(best_eff / 100, 1e-10)):.2f} dB) at step {best_idx + 1}")
theta_final = results[-1]['theta']

## 8. Results

In [ ]:
powers = [abs(r['loss']) for r in results]
efficiencies = [p / input_power * 100 for p in powers]
best_idx = int(np.argmax(efficiencies))
best_theta = results[best_idx]['theta']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Efficiency curve
axes[0].plot(range(1, len(efficiencies) + 1), efficiencies, 'b-o', markersize=3)
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Coupling Efficiency (%)')
axes[0].set_title('Waveguide Coupling Efficiency')
axes[0].grid(True, alpha=0.3)

# Initial vs best theta
extent = [0, theta_Lx * pixel_size, 0, theta_Ly * pixel_size]
for ax, th, title in [(axes[1], theta_init, 'Initial'),
                       (axes[2], best_theta, f'Best (step {best_idx + 1})')]:
    im = ax.imshow(th.T, origin='lower', cmap='PuOr', vmin=0, vmax=1, extent=extent)
    ax.set_xlabel('x (um)')
    ax.set_ylabel('y (um)')
    ax.set_title(title)
    plt.colorbar(im, ax=ax)
    rect = Rectangle(
        (dr['x_start'] * pixel_size, dr['y_start'] * pixel_size),
        (dr['x_end'] - dr['x_start']) * pixel_size,
        (dr['y_end'] - dr['y_start']) * pixel_size,
        linewidth=1.5, edgecolor='lime', facecolor='none', linestyle='--',
    )
    ax.add_patch(rect)

plt.tight_layout()
plt.show()

## 9. Optimized Design Field Plots

Run forward simulation with the best design to visualize field distribution
and verify coupling into the waveguide.

In [ ]:
# Build recipe for optimized theta (no 3D array)
density_best = hwc.density(theta=jnp.array(best_theta), pad_width=0, alpha=DENSITY_ALPHA, radius=DENSITY_RADIUS)
theta_bottom_best = np.where(best_theta > 0, 1.0, 0.0).astype(np.float32)
density_bottom_best = hwc.density(theta=jnp.array(theta_bottom_best), pad_width=0, alpha=DENSITY_ALPHA, radius=DENSITY_RADIUS)

recipe_best = hwc.recipe_from_params(
    grid_shape=density_best.shape,
    layers=[
        {'density': np.array(slab_density),        'permittivity': eps_air,            'thickness': h_p},
        {'density': np.array(slab_density),        'permittivity': eps_air,            'thickness': h0},
        {'density': np.array(slab_density),        'permittivity': eps_clad,           'thickness': h1},
        {'density': np.array(density_best),        'permittivity': (eps_clad, eps_si), 'thickness': h2},
        {'density': np.array(density_bottom_best), 'permittivity': (eps_clad, eps_si), 'thickness': h3},
        {'density': np.array(slab_density),        'permittivity': eps_sio2,           'thickness': h4},
        {'density': np.array(slab_density),        'permittivity': eps_si,             'thickness': h5},
        {'density': np.array(slab_density),        'permittivity': eps_si,             'thickness': h_p},
    ],
    vertical_radius=0,
)

# Forward simulation with optimized design
opt_results = hwc.simulate(
    structure_recipe=recipe_best,
    source_field=source_field,
    source_offset=source_offset,
    freq_band=freq_band,
    monitors_recipe=monitors.recipe,
    absorption_widths=abs_widths,
    absorption_coeff=abs_coeff,
    gpu_type="B200",
)

# Plot |E|^2 cross-sections
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for ax, name, title, ext, xlabel, ylabel in [
    (axes[0, 0], 'xy_device', f'|E|^2 XY (z = {z_dev * dx:.2f} um)',
     [0, Lx * dx, 0, Ly * dx], 'x (um)', 'y (um)'),
    (axes[0, 1], 'xz_center', '|E|^2 XZ (y = center)',
     [0, Lx * dx, 0, Lz * dx], 'x (um)', 'z (um)'),
    (axes[0, 2], 'yz_center', '|E|^2 YZ (x = center)',
     [0, Ly * dx, 0, Lz * dx], 'y (um)', 'z (um)'),
]:
    field = np.array(opt_results['monitor_data'][name])
    E2 = np.sum(np.abs(field[0, 0:3])**2, axis=0).squeeze()
    vmax = np.percentile(E2, 95) * 4
    im = ax.imshow(E2.T, origin='lower', cmap='hot', extent=ext,
                   aspect='auto', vmax=vmax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.colorbar(im, ax=ax, extend='max')

field_xy = np.array(opt_results['monitor_data']['xy_device'])
for i, name in enumerate(['|Ex|', '|Ey|', '|Ez|']):
    comp = np.abs(field_xy[0, i, :, :, 0])
    im = axes[1, i].imshow(comp.T, origin='lower', cmap='hot',
                            extent=[0, Lx * dx, 0, Ly * dx])
    axes[1, i].set_xlabel('x (um)')
    axes[1, i].set_ylabel('y (um)')
    axes[1, i].set_title(name)
    plt.colorbar(im, ax=axes[1, i])

plt.suptitle('Optimized design fields', fontsize=14)
plt.tight_layout()
plt.show()

# Coupling efficiency from Poynting vector power
wg_field = np.array(opt_results['monitor_data']['wg_output'])
S = hwc.S_from_slice(jnp.mean(jnp.array(wg_field), axis=2))
wg_power = float(jnp.abs(jnp.sum(S[:, 0, :, :], axis=(1, 2))))
eff_pct = wg_power / input_power * 100
loss_dB = -10 * np.log10(max(eff_pct / 100, 1e-10))
print(f"Coupling efficiency: {eff_pct:.2f}% ({loss_dB:.2f} dB)")

## Next Steps

**This is a preview of the inverse design workflow.** The optimized design
contains grayscale (intermediate) density values and has not been processed
for fabrication. Before manufacturing, the following steps are needed:

- **Binarization:** Increase `density_alpha` gradually from 0 to 1 over the
  course of optimization. At `alpha=0` the design is fully freeform (grayscale
  allowed); at `alpha=1` it is fully binarized (every pixel pushed to 0 or 1).
  A good strategy: run 50 steps with `alpha=0` to find a good design, then
  continue for 50 more steps while linearly ramping alpha to 1.
- **Fabrication constraints:** Enforce minimum feature size, minimum spacing,
  and etch design rules for the target foundry process (in development).
- **GDS export:** Only after binarization and constraint enforcement, convert
  to GDSII using `hwc.generate_gds_from_density()`.

**Improving results:**
- **More steps:** Increase `NUM_STEPS` to 50-100. Grating couplers typically
  need 50+ steps to converge; gains slow down but don't fully plateau until
  100-200 steps.
- **Mode overlap loss:** For higher accuracy, use `mode_field` parameter with a
  pre-computed waveguide mode instead of `power_axis`. This optimizes for
  coupling into the fundamental waveguide mode specifically, rather than total
  power. See `hwc.solve_mode_source()` for cloud-based mode solving.
- **Different starting points:** Adjoint optimization uses gradient descent,
  which converges to a local optimum -- not necessarily the global best. The
  loss landscape for photonic inverse design is non-convex, so different
  initial theta values can lead to different final designs. Try several random
  initializations (e.g., `theta_init = 0.3`, `0.5`, `0.7`, or random patterns)
  and keep the best result.
- **Finer grid resolution:** Decrease `dx` (e.g., 25nm or 20nm) for more
  accurate results. Finer grids resolve sub-wavelength features better but
  increase simulation cost quadratically.
- **Multi-frequency optimization:** Expand `freq_band` to multiple wavelengths
  (e.g., 1530-1570nm) to optimize for bandwidth instead of a single wavelength.

**Design parameters:**
- `density_radius` controls minimum feature size via a conical blur filter.
  `radius=3` at 35nm grid enforces roughly 100nm minimum features.
- `density_alpha` controls binarization strength. Start at 0 for exploration,
  increase toward 1 to push the design to fabricable binary values.
- The optimizer (Adam with cosine learning rate decay and gradient clipping) is
  built into `run_optimization` and does not need manual tuning for most cases.